In [1]:
BATCH_SIZE = 64
N_CHANNELS = 5

In [2]:
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import torch
from network_definitions.u_net import UNet
from network_definitions.fcn import FCN32s as FCN
from network_definitions.simple_network import SimpleNet
from network_definitions.pyramid_network import PyramidNet
from torchvision.models.segmentation import fcn_resnet101 as FCN_Res101

# Dataset Import

In [3]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

class EnsembleDataset(Dataset):
    """Ensemble dataset."""

    def __init__(self, results_file, transform=None):
        """
        Args:
            results_file (string): File with all the results.
        """
        with open(results_file, 'rb') as f:
            #compressed_file = bz2.BZ2File(f, 'r')
            self.results = pickle.load(f)
        self.transform = transform

    def __len__(self):
        return len(self.results)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.results[idx]
        sample = {'name': data[0], 'valid': data[1], 'im_seg': data[2], 'im_res': data[3]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [4]:
from skimage.transform import resize
from torchvision import transforms, utils

class Resize(object):
    def __init__(self, size):
        self.size = size

    def __call__(self,sample):
        name,valid,im_seg,im_res = sample["name"],sample["valid"],sample["im_seg"],sample["im_res"]
        
        return {"name": name, "valid": valid, "im_seg": resize(im_seg,(self.size,self.size,N_CHANNELS),preserve_range=True), "im_res": resize(im_res,(self.size,self.size,1),preserve_range=True)}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        name,valid,im_seg,im_res = sample["name"],sample["valid"],sample["im_seg"],sample["im_res"]

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        im_seg = im_seg.transpose((2, 0, 1))
        im_res = im_res.transpose((2, 0, 1))
        return {"name": name, 
                "valid": valid,
                "im_seg": torch.from_numpy(im_seg),
                "im_res": torch.from_numpy(im_res)}

In [5]:
trainset = EnsembleDataset(results_file='work_dirs/dataset_generation/dataset_no_img.pkl', 
                           transform=transforms.Compose([Resize(572),
                                                         ToTensor()]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=6)

"""trainset = EnsembleDataset(image_dir='data/coco/test2017',
                           results_file='',
                           transform=transforms.Compose([Rescale(256),
                                                         RandomCrop(224),
                                                         ToTensor()]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)"""



"trainset = EnsembleDataset(image_dir='data/coco/test2017',\n                           results_file='',\n                           transform=transforms.Compose([Rescale(256),\n                                                         RandomCrop(224),\n                                                         ToTensor()]))\ntrainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,\n                                          shuffle=True, num_workers=2)"

# Training

In [10]:
from torch.utils.tensorboard import SummaryWriter

#PATH = "work_dirs/simplenet_1/"


def train(net, trainloader, criterion, optimizer, save_path, tensorboard_path, checkpoint=None):
    
    EPOCH = 0
    
    writer = SummaryWriter(log_dir=tensorboard_path)
    
    if checkpoint != None:
        checkpoint = torch.load(checkpoint)
        net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        EPOCH = checkpoint['epoch']
        loss = checkpoint['loss']
        net.train()
    
    for epoch in range(EPOCH,25):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            im_seg = data["im_seg"].to(device, dtype=torch.float)
            im_res = data["im_res"].to(device, dtype=torch.float)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            output = net(im_seg.float())
            loss = criterion(output, im_res.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 50 == 49:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.6f' %
                      (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0
                inp = im_seg.cpu().detach()
                output = output.cpu().detach()
                gt_output = im_res.cpu().detach()
                
                inp = inp.numpy()[0].transpose((1,2,0))
                #.squeeze(axis=0)
                output = output.numpy()[0].transpose((1,2,0)).squeeze(axis=2)
                gt_output = gt_output.numpy()[0].transpose((1,2,0)).squeeze(axis=2)
                
                fig, ax = plt.subplots(nrows=1, ncols=7, figsize=(15,15))
                ax=ax.flat
                
                for i in range(0,5):
                    #ax.append(fig.add_subplot(2, 4, i+1))
                    ax[i].set_title("Input "+str(i+1))  # set title
                    ax[i].imshow(inp[:,:,i],cmap='gray',vmin=0,vmax=1)
                    
                
                    
                #ax.append(fig.add_subplot(2, 4, 6))
                ax[5].set_title("Output")  # set title
                ax[5].imshow(output,cmap='gray',vmin=0,vmax=1)
                
                #ax.append(fig.add_subplot(2, 4, 7))
                ax[6].set_title("GT Output")  # set title
                ax[6].imshow(gt_output,cmap='gray',vmin=0,vmax=1)
                
                fig.tight_layout()

                plt.show()
                
                print(inp[:,:,i].max(),inp[:,:,i].min())
                print(output.max(),output.min())
            
        writer.add_scalar('Loss', loss, epoch)

        if epoch % 5 == 4:        
            torch.save({
                'epoch': epoch,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, save_path+"epoch_"+str(epoch+1)+".pt")
    
    writer.close()

    print('Finished Training')

In [11]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ACTIVATION = "sigmoid"
LOSS = "BCELoss"

for layers in [1,2]:
    print("Starting training on network ",layers)
    
    net = PyramidNet(N_CHANNELS,layers,activation=ACTIVATION)
    net = net.to(device).float()
    if LOSS == "BCELoss":
        criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    
    checkpoint_path = "work_dirs/pyramidnet"
    #for layer in layers:
    checkpoint_path += "_"+str(layers)
    checkpoint_path += "/" + ACTIVATION + "_" + LOSS + "/"
    tensorboard_path = checkpoint_path+"tb/"
    os.makedirs(tensorboard_path,exist_ok=True)
    
    train(net,trainloader,criterion, optimizer, checkpoint_path, tensorboard_path)#, checkpoint="work_dirs/simplenet_1/epoch_25.pt")

Starting training on network  1


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
event_acc = EventAccumulator('work_dirs/simplenet_1_1_1/sigmoid_BCELoss/tb')
event_acc.Reload()
# Show all tags in the log file
print(event_acc.Tags())

# E. g. get wall clock, number of steps and value for a scalar 'Accuracy'
w_times, step_nums, vals = zip(*event_acc.Scalars('Loss'))

# Network Summary

In [8]:
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PyramidNet(5,1,activation="sigmoid").float().to(device)

summary(model, (1,5,572,572))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
# for i in range(1):
    data = trainset[i]
    
    im_seg = data['im_seg']
    im_res = data['im_res']
    
    res = im_seg[0:3,:,:].numpy().transpose((1,2,0))
    
    fig = plt.figure()
    plt.imshow(res)